In [ ]:
!pip install tvm_valuetypes

In [1]:
#%load_ext autoreload
#%autoreload 2
from tvm_python import PyTVM, method_name_to_id, code_disasseble
import codecs
from tvm_valuetypes import Cell

# Just code

In [2]:
PyTVM(code="te6ccgEBAQEAAwAAAnE=").run_vm()

[1]

# Disassembler

In [3]:
code_disasseble("te6ccgEBAQEAAwAAAnE=")

'PUSHINT 1\n'

# Easy test examples

In [ ]:
# Simple PUSHINT 1
# You can define log_level if you want more stuff
tvm_result = PyTVM(code="te6ccgEBAQEAAwAAAnE=", log_level=4).run_vm()
assert tvm_result[0] == 1

print(f"Got 1 from TVM: {tvm_result[0]}")

In [ ]:
tvm_result

In [ ]:
# c3 is continuation, but we just serialize it to base64 BOC
# PUSHINT 1 PUSH c3
tvm_result = PyTVM(code="te6ccgEBAQEABQAABnHtQw==").run_vm()
tvm_result

In [ ]:
cell = Cell()
b64 = tvm_result[0]['value']
cell.data.from_bytes(codecs.decode(codecs.encode(b64, 'utf8'), 'base64'))

cell.data # This is continuation

In [ ]:
# c7 push (tuple)
tvm_result = PyTVM(code="te6ccgEBAQEABAAABO1H").run_vm()
tvm_result[0] # just default c7

## Change c7

In [ ]:
# c7 push (tuple)
tvm = PyTVM(code="te6ccgEBAQEABAAABO1H")

In [ ]:
tvm.set_c7(translt=100,
           unixtime=179, 
           globalConfig="te6ccgEBAQEABAAABO1H",
           randseed=100, 
           address="EQBq5JLG8-1juc-N95IFBnOM2NGrcJsMoFYFoWH-DaVJ2lVw",
           balanceGrams=100000000)

In [ ]:
tvm.run_vm()

## Get detailed result

In [ ]:
print("Exit code: ", tvm.exit_code,
"\nC5: ", tvm.actions, 
"\nGas credit: ",tvm.gas_credit, 
"\nGas used: ",tvm.gas_used, 
"\nC4: ",tvm.new_data, 
"\nVM Steps: ",tvm.vm_steps,
"\nVM final hash: ",tvm.vm_final_state_hash,
"\nVM init hash: ",tvm.vm_init_state_hash,
"\nVM success: ",tvm.success)

## Set stack

In [ ]:
# c7 push (tuple)
tvm = PyTVM(code="te6ccgEBAQEABAAABO1H", log_level=6)

In [ ]:
# actually python3 & TON have really big integers :))
tvm.set_stack([
    190009329293929392939239293293929329392949949949299008888888880999999888, 
    {'type': "cell",
     'value': 'te6ccgEBAQEABAAABO1H'}, 
    [{'type': "cellSlice",
     'value': 'te6ccgEBAQEABAAABO1H'},
     12312312313213132,
     [1231232132, 1321231231, 57, 179]
    ]])

In [ ]:
tvm.run_vm()

## C4 (Data) usage example

In [ ]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=2)

In [ ]:
# You can see, that 179 was loaded from c4, and cellSlice on stack
tvm.run_vm()

## Gas Limit 

In [ ]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=2)

In [ ]:
tvm.set_gasLimit(26) # You can also pass max gas set_gasLimit(gas_limit=26, gas_max=26)
tvm.run_vm() # out of gas

## Libs 

This is just load example, they will pass to VM, but I'm little bussy to get code of loading libs, check out [akifoq](https://github.com/akifoq) github

In [ ]:
tvm.set_libs([{'type': 'cell', 'value': 'te6ccgEBAQEABwAACu1E0NM/'}])

## Stack dump log Level

In [ ]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=4)

In [ ]:
tvm.run_vm()

## Allow debug & same c3

In [ ]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=4,
            allow_debug=True,
            same_c3=False)

In [ ]:
tvm.run_vm()

## Accout get methods

In [ ]:
# get account code & data from dton.io
tvm = PyTVM(code="te6ccuECFAEAAtQAABoAJAAuATICIAIqAqYDNAM+A5wDpgO8A/4ECAQmBEQEtgUqBZoFqAEU/wD0pBP0vPLICwECASACAwIBSAQFBPjygwjXGCDTH9Mf0x8C+CO78mTtRNDTH9Mf0//0BNFRQ7ryoVFRuvKiBfkBVBBk+RDyo/gAJKTIyx9SQMsfUjDL/1IQ9ADJ7VT4DwHTByHAAJ9sUZMg10qW0wfUAvsA6DDgIcAB4wAhwALjAAHAA5Ew4w0DpMjLHxLLH8v/EBESEwLm0AHQ0wMhcbCSXwTgItdJwSCSXwTgAtMfIYIQcGx1Z70ighBkc3RyvbCSXwXgA/pAMCD6RAHIygfL/8nQ7UTQgQFA1yH0BDBcgQEI9ApvoTGzkl8H4AXTP8glghBwbHVnupI4MOMNA4IQZHN0crqSXwbjDQYHAgEgCAkAeAH6APQEMPgnbyIwUAqhIb7y4FCCEHBsdWeDHrFwgBhQBMsFJs8WWPoCGfQAy2kXyx9SYMs/IMmAQPsABgCKUASBAQj0WTDtRNCBAUDXIMgBzxb0AMntVAFysI4jghBkc3Rygx6xcIAYUAXLBVADzxYj+gITy2rLH8s/yYBA+wCSXwPiAgEgCgsAWb0kK29qJoQICga5D6AhhHDUCAhHpJN9KZEM5pA+n/mDeBKAG3gQFImHFZ8xhAIBWAwNABG4yX7UTQ1wsfgAPbKd+1E0IEBQNch9AQwAsjKB8v/ydABgQEI9ApvoTGACASAODwAZrc52omhAIGuQ64X/wAAZrx32omhAEGuQ64WPwABu0gf6ANTUIvkABcjKBxXL/8nQd3SAGMjLBcsCIs8WUAX6AhTLaxLMzMlz+wDIQBSBAQj0UfKnAgBwgQEI1xj6ANM/yFQgR4EBCPRR8qeCEG5vdGVwdIAYyMsFywJQBs8WUAT6AhTLahLLH8s/yXP7AAIAbIEBCNcY+gDTPzBSJIEBCPRZ8qeCEGRzdHJwdIAYyMsFywJQBc8WUAP6AhPLassfEss/yXP7AAAK9ADJ7VQ32i8B", 
            data="te6ccuEBBAEAegBYZKz0AVEAAABfKamjFwar2UnSv7yFJMcBHp8S/8es5VZ1DSsQduQyjHhdbU9awAECA8CIAgMAQ7/9RH6C0XtQV/uceTSwYGqZEYjcVUKe3D2prT8KNc4u5EAAQ7/Nan0TcqBqB20NuDhtNOk60TWn6tZwV5g88o9fm8cwK8CwqgZg", 
            log_level=4)

In [ ]:
tvm.set_c7(address="EQCI5yVsMccJDqrKH9knYAp0avK5fh-UBZ5Ns1xWOvIA2UKl", # some contracts may use MYADDR in code
           balanceGrams=100000000, # or trying get own balance from c7
           randseed=1) # or maybe you use random in your contract :)

In [ ]:
method_name_to_id("get_seqno") # get methods have own numbers :)

In [ ]:
tvm.set_stack([method_name_to_id("seqno")]) # long story short, you need to pass method_id to stack

In [ ]:
tvm.run_vm() # this is seqno of this wallet :)